## Importações

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import shap
import seaborn as sns

# Pré-processamento e pipelines
from transformers_credits import DateFeatureExtractor, CreditsCounter, MultiLabelProcessor, RareCategoryGrouper, CapTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Modelos de Regressão
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

# Validação e busca de hiperparâmetros
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from scipy.stats import randint, uniform, loguniform

# Métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

c:\temp\ws-vscode\DSI-PISI3\projeto-pisi3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carregando dataset

In [2]:
df = pd.read_csv('../data/filmes_filtrados_credits.csv')

## Dividindo os dados

In [3]:
# Definição das colunas por tipo
TARGET = 'vote_average'
DATE_COLUMN = 'release_date'
CREDITS_COL = 'credits'
MULTILABEL_COLS_CONFIG = {
    'genres': {'sep': '-', 'top_n': 12, 'prefix': 'genre'},
    'production_companies': {'sep': '-', 'top_n': 8, 'prefix': 'prod'}
}
NUMERICAL_COLS_RAW = ['popularity', 'budget', 'runtime'] # Colunas numéricas originais
CATEGORICAL_COL = ['original_language']

# Colunas que entrarão no modelo
features = NUMERICAL_COLS_RAW + CATEGORICAL_COL + [DATE_COLUMN, CREDITS_COL] + list(MULTILABEL_COLS_CONFIG.keys())

X = df[features]
y = df[TARGET]

# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

## Construção do Pipeline de Pré-processamento

In [4]:
# O pipeline principal irá gerar novas colunas. Definimos os nomes aqui para o ColumnTransformer
NUMERICAL_COLS_FINAL = NUMERICAL_COLS_RAW + ['credits_count']

# Pipeline para colunas numéricas (Cap de outliers + Padronização)
numeric_transformer = Pipeline(steps=[
    ('cap', CapTransformer(columns=NUMERICAL_COLS_FINAL)),
    ('scaler', StandardScaler())
])

# Pipeline para a coluna categórica (Agrupamento de raras + OneHot)
categorical_transformer = Pipeline(steps=[
    ('rare', RareCategoryGrouper(column='original_language', top_n=10)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pré-processador que aplica transformações específicas a cada tipo de coluna
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, NUMERICAL_COLS_FINAL),
        ('cat', categorical_transformer, CATEGORICAL_COL)
    ],
    remainder='passthrough' # Mantém as colunas processadas anteriormente (ano, mês, multilabel)
)

# Pipeline completo que encadeia todas as etapas de engenharia de features e pré-processamento
features_pipeline = Pipeline(steps=[
    ('credits_transform', CreditsCounter(credits_column=CREDITS_COL)),
    ('date_transform', DateFeatureExtractor(date_column=DATE_COLUMN)),
    ('multilabel_transform', MultiLabelProcessor(multilabel_cols_config=MULTILABEL_COLS_CONFIG)),
    ('preprocess', preprocessor)
])

Célula 1: Definição do Pipeline e KFold


In [5]:
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

# Pipeline de pré-processamento (definido em células anteriores)
# features_pipeline = Pipeline(...) 

# KFold para validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=87)

# Dicionários para armazenar os resultados e os melhores modelos
results = {}
best_estimators = {}

Célula 2: Treinamento do XGBoost


In [6]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

print("--- Treinando e otimizando XGBoost ---")

# Pipeline específico para o XGBoost
xgb_pipeline = Pipeline(steps=[
    ('features', features_pipeline),
    ('regressor', XGBRegressor(objective='reg:squarederror', random_state=87, n_jobs=-1))
])

# Espaço de busca de hiperparâmetros para o XGBoost
xgb_params = {
    'regressor__n_estimators': randint(100, 300),
    'regressor__max_depth': randint(3, 10),
    'regressor__learning_rate': uniform(0.01, 0.2),
    'regressor__subsample': uniform(0.7, 0.3),
    'regressor__colsample_bytree': uniform(0.7, 0.3),
}

# Busca randomizada
xgb_rand_search = RandomizedSearchCV(
    estimator=xgb_pipeline,
    param_distributions=xgb_params,
    n_iter=25,
    cv=kf,
    scoring='r2',
    random_state=87,
    verbose=1,
    n_jobs=-1
)

xgb_rand_search.fit(X_train, y_train)
best_estimators['XGBoost'] = xgb_rand_search.best_estimator_

print(f"Melhores parâmetros para XGBoost: {xgb_rand_search.best_params_}\n")
print(f"Melhor R² (CV) para XGBoost: {xgb_rand_search.best_score_}")

--- Treinando e otimizando XGBoost ---
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Melhores parâmetros para XGBoost: {'regressor__colsample_bytree': np.float64(0.8696231377727208), 'regressor__learning_rate': np.float64(0.04328037357016285), 'regressor__max_depth': 5, 'regressor__n_estimators': 241, 'regressor__subsample': np.float64(0.8173221947525602)}

Melhor R² (CV) para XGBoost: 0.4929762682543406


Célula 3: Treinamento do RandomForest


In [7]:
from sklearn.ensemble import RandomForestRegressor

print("--- Treinando e otimizando RandomForest ---")

# Pipeline específico para o RandomForest
rf_pipeline = Pipeline(steps=[
    ('features', features_pipeline),
    ('regressor', RandomForestRegressor(random_state=87, n_jobs=-1))
])

# Espaço de busca para o RandomForest
rf_params = {
    'regressor__n_estimators': randint(100, 500),
    'regressor__max_depth': randint(5, 20),
    'regressor__min_samples_leaf': randint(1, 10),
}

# Busca randomizada
rf_rand_search = RandomizedSearchCV(
    estimator=rf_pipeline,
    param_distributions=rf_params,
    n_iter=25,
    cv=kf,
    scoring='r2',
    random_state=87,
    verbose=1,
    n_jobs=-1
)

rf_rand_search.fit(X_train, y_train)
best_estimators['RandomForest'] = rf_rand_search.best_estimator_

print(f"Melhores parâmetros para RandomForest: {rf_rand_search.best_params_}\n")
print(f"Melhor R² (CV) para RandomForest: {rf_rand_search.best_score_}")

--- Treinando e otimizando RandomForest ---
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Melhores parâmetros para RandomForest: {'regressor__max_depth': 19, 'regressor__min_samples_leaf': 3, 'regressor__n_estimators': 354}

Melhor R² (CV) para RandomForest: 0.46663531375549433


Célula 4: Treinamento do SVR


In [8]:
from sklearn.svm import SVR
from scipy.stats import loguniform

print("--- Treinando e otimizando SVR ---")

# Pipeline específico para o SVR
svr_pipeline = Pipeline(steps=[
    ('features', features_pipeline),
    ('regressor', SVR())
])

# Espaço de busca para o SVR
svr_params = {
    'regressor__C': loguniform(1e-1, 1e2),
    'regressor__epsilon': uniform(0.01, 0.3),
    'regressor__gamma': ['scale', 'auto']
}

# Busca randomizada
svr_rand_search = RandomizedSearchCV(
    estimator=svr_pipeline,
    param_distributions=svr_params,
    n_iter=25,
    cv=kf,
    scoring='r2',
    random_state=87,
    verbose=1,
    n_jobs=-1
)

svr_rand_search.fit(X_train, y_train)
best_estimators['SVR'] = svr_rand_search.best_estimator_

print(f"Melhores parâmetros para SVR: {svr_rand_search.best_params_}\n")
print(f"Melhor R² (CV) para SVR: {svr_rand_search.best_score_}")

--- Treinando e otimizando SVR ---
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Melhores parâmetros para SVR: {'regressor__C': np.float64(1.1395264134607888), 'regressor__epsilon': np.float64(0.24739505322469346), 'regressor__gamma': 'auto'}

Melhor R² (CV) para SVR: 0.442824040410271


Célula 5: Avaliação Final dos Modelos


In [9]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np

def evaluate_model(model_name, estimator, X_test, y_test):
    """Função para avaliar um modelo e retornar um dicionário de métricas."""
    y_pred = estimator.predict(X_test)
    return {
        'R² (Teste)': r2_score(y_test, y_pred),
        'MAE (Teste)': mean_absolute_error(y_test, y_pred),
        'RMSE (Teste)': np.sqrt(mean_squared_error(y_test, y_pred)),
    }

# Avalia cada um dos melhores modelos
results['XGBoost'] = evaluate_model('XGBoost', best_estimators['XGBoost'], X_test, y_test)
results['RandomForest'] = evaluate_model('RandomForest', best_estimators['RandomForest'], X_test, y_test)
results['SVR'] = evaluate_model('SVR', best_estimators['SVR'], X_test, y_test)

# Adiciona a pontuação de validação cruzada para comparação
results['XGBoost']['R² (CV)'] = xgb_rand_search.best_score_
results['RandomForest']['R² (CV)'] = rf_rand_search.best_score_
results['SVR']['R² (CV)'] = svr_rand_search.best_score_

# Cria e exibe o DataFrame de resultados
results_df = pd.DataFrame(results).T
results_df = results_df[['R² (CV)', 'R² (Teste)', 'MAE (Teste)', 'RMSE (Teste)']] # Reordena colunas
results_df = results_df.sort_values(by='R² (Teste)', ascending=False)

print("\n--- Resultados Finais da Avaliação ---")
display(results_df)


--- Resultados Finais da Avaliação ---


,R² (CV),R² (Teste),MAE (Teste),RMSE (Teste)
XGBoost,0.492976,0.487283,0.498563,0.652966
RandomForest,0.466635,0.458910,0.512332,0.670791
SVR,0.442824,0.442009,0.515798,0.681186
